# 01_preparation — Nettoyage & préparation (Online Retail II)
**But :** produire un jeu de données transactionnel propre et une table clients (RFM) utilisables par l'application Streamlit du projet.

**Fichier source (local) :** `online_retail_II.xlsx`  
**Sorties attendues :**
- `clean_data/transactions_clean.parquet`
- `clean_data/transactions_clean.xlsx`
- `clean_data/customers_clean.parquet`
- `clean_data/customers_clean.xlsx`
- `clean_data/qc_report.json`

**Plan :**
1. Chargement & examen
2. Profiling & qualité
3. Nettoyage pas à pas
4. Enrichissements temporels & cohortes
5. Construction RFM
6. Exports & rapport QC






In [ ]:
!pip install openpyxl


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Importation des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path

# Options d'affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f"{x:.2f}")

# Paramètres / chemins d'accès
file_path = "online_retail_II.xlsx"
OUT_DIR = Path("clean_data")
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("Fichier source attendu :", file_path)
print("Sorties dans :", OUT_DIR.resolve())


Fichier source attendu : online_retail_II.xlsx
Sorties dans : C:\Users\DELL\Downloads\projet data viz\clean_data


In [ ]:
# Lecture : l'Excel peut contenir plusieurs feuilles. On liste d'abord.
try:
    xls = pd.ExcelFile(file_path, engine='openpyxl')
    print("Sheets trouvées :", xls.sheet_names)
except Exception as e:
    raise RuntimeError(f"Impossible d'ouvrir le fichier Excel : {e}")

# Si plusieurs sheets, on concatène ; si une seule, on la charge.
frames = []
for sheet in xls.sheet_names:
    print(f"Lecture de la feuille : {sheet} ...")
    df_sheet = pd.read_excel(file_path, sheet_name=sheet, engine='openpyxl')
    df_sheet['__source_sheet'] = sheet
    frames.append(df_sheet)

df = pd.concat(frames, ignore_index=True)
print("Shape après concat :", df.shape)
df.head()


Sheets trouvées : ['Year 2009-2010', 'Year 2010-2011']
Lecture de la feuille : Year 2009-2010 ...
Lecture de la feuille : Year 2010-2011 ...
Shape après concat : (1067371, 9)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,__source_sheet
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom,Year 2009-2010
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom,Year 2009-2010
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom,Year 2009-2010
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom,Year 2009-2010
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom,Year 2009-2010


## Remarques méthodologiques
- On conservera une version *raw* (non modifiée) sauvegardée localement si tu veux garder un backup.
- Ici on travaille sur une copie `df` en mémoire ; les transformations sont appliquées sur cette copie.
- Toutes les étapes importantes seront loggées et résumées dans `qc_report.json`.


In [ ]:
# Profiling initial rapide
def basic_profile(df):
    profile = {
        'n_rows': int(len(df)),
        'n_cols': int(df.shape[1]),
        'columns': df.columns.tolist(),
        'dtypes': {c: str(dt) for c, dt in df.dtypes.items()},
        'missing_per_col': df.isnull().sum().to_dict()
    }
    # period if invoice date exists
    if 'InvoiceDate' in df.columns:
        try:
            profile['invoice_min'] = str(pd.to_datetime(df['InvoiceDate'], dayfirst=True, errors='coerce').min())
            profile['invoice_max'] = str(pd.to_datetime(df['InvoiceDate'], dayfirst=True, errors='coerce').max())
        except Exception as e:
            profile['invoice_error'] = str(e)
    return profile

profile0 = basic_profile(df)
print(json.dumps(profile0, indent=2))


{
  "n_rows": 1067371,
  "n_cols": 9,
  "columns": [
    "Invoice",
    "StockCode",
    "Description",
    "Quantity",
    "InvoiceDate",
    "Price",
    "Customer ID",
    "Country",
    "__source_sheet"
  ],
  "dtypes": {
    "Invoice": "object",
    "StockCode": "object",
    "Description": "object",
    "Quantity": "int64",
    "InvoiceDate": "datetime64[ns]",
    "Price": "float64",
    "Customer ID": "float64",
    "Country": "object",
    "__source_sheet": "object"
  },
  "missing_per_col": {
    "Invoice": 0,
    "StockCode": 0,
    "Description": 4382,
    "Quantity": 0,
    "InvoiceDate": 0,
    "Price": 0,
    "Customer ID": 243007,
    "Country": 0,
    "__source_sheet": 0
  },
  "invoice_min": "2009-12-01 07:45:00",
  "invoice_max": "2011-12-09 12:50:00"
}


In [ ]:
# Quelques affichages utiles : types, top countries, missing CustomerID
print("Types :")
print(df.dtypes)
print("\nTop 10 pays :")
if 'Country' in df.columns:
    print(df['Country'].value_counts().head(10))
else:
    print("Aucune colonne 'Country' trouvée.")

if 'CustomerID' in df.columns:
    n_missing_cust = df['CustomerID'].isnull().sum()
    print(f"\nCustomerID missing : {n_missing_cust} / {len(df)} ({n_missing_cust/len(df):.2%})")
else:
    print("\nAucune colonne 'CustomerID' trouvée.")


Types :
Invoice                   object
StockCode                 object
Description               object
Quantity                   int64
InvoiceDate       datetime64[ns]
Price                    float64
Customer ID              float64
Country                   object
__source_sheet            object
dtype: object

Top 10 pays :
Country
United Kingdom    981330
EIRE               17866
Germany            17624
France             14330
Netherlands         5140
Spain               3811
Switzerland         3189
Belgium             3123
Portugal            2620
Australia           1913
Name: count, dtype: int64

Aucune colonne 'CustomerID' trouvée.


## Stratégie de nettoyage (liste)
- Standardiser noms de colonnes (strip)
- Nettoyer chaînes (strip, remplacer valeurs vides)
- Convertir InvoiceDate en datetime (dayfirst=True)
- Forcer Quantity en entier, UnitPrice en float
- Calculer Amount = Quantity * UnitPrice
- Marquer retours (InvoiceNo commençant par 'C')
- Filtrer Quantity == 0
- Supprimer doublons stricts sur colonnes clés
- Marquer outliers sur Quantity / UnitPrice
- Séparer datasets :
  - transactions_including_returns
  - transactions_excluding_returns
  - transactions_customers (CustomerID not null)
- Construire RFM sur transactions_customers


In [ ]:
# 1) Standardiser noms colonnes
df.columns = [c.strip() if isinstance(c, str) else c for c in df.columns]

# 2) Nettoyage strings sur colonnes attendues
for c in ['InvoiceNo', 'StockCode', 'Description', 'Country']:
    if c in df.columns:
        df[c] = df[c].astype('string').str.strip()
    else:
        print(f"Colonne manquante : {c} (sera créée si nécessaire)")

# Affichage après nettoyage
df.head(3)


Colonne manquante : InvoiceNo (sera créée si nécessaire)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,__source_sheet
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom,Year 2009-2010
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom,Year 2009-2010
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom,Year 2009-2010


In [ ]:
# Conversion InvoiceDate
if 'InvoiceDate' in df.columns:
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], dayfirst=True, errors='coerce')
    n_nat = df['InvoiceDate'].isna().sum()
    print(f"InvoiceDate -> datetime ; NaT count: {n_nat}")
else:
    raise KeyError("InvoiceDate absent du dataset — vérifie le fichier source")

# Quantity -> numeric (entier)
if 'Quantity' in df.columns:
    df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce').fillna(0).astype(int)
else:
    df['Quantity'] = 0

# UnitPrice -> numeric float
if 'UnitPrice' in df.columns:
    df['UnitPrice'] = pd.to_numeric(df['UnitPrice'], errors='coerce').fillna(0.0)
else:
    df['UnitPrice'] = 0.0

df[['InvoiceDate','Quantity','UnitPrice']].head()


InvoiceDate -> datetime ; NaT count: 0


,InvoiceDate,Quantity,UnitPrice
0,2009-12-01 07:45:00,12,0.00
1,2009-12-01 07:45:00,12,0.00
2,2009-12-01 07:45:00,12,0.00
3,2009-12-01 07:45:00,48,0.00
4,2009-12-01 07:45:00,24,0.00


In [ ]:
# Marquer retours : InvoiceNo commençant par C
df['is_cancel'] = False
if 'InvoiceNo' in df.columns:
    df['is_cancel'] = df['InvoiceNo'].astype(str).str.startswith('C')

# Calculer Amount
df['Amount'] = df['Quantity'].astype(float) * df['UnitPrice'].astype(float)

# Vérifications rapides
print("Retours (is_cancel) counts :")
print(df['is_cancel'].value_counts(dropna=False))
print("\nAmount sample :")
print(df[['Quantity','UnitPrice','Amount']].head())


Retours (is_cancel) counts :
is_cancel
False    1067371
Name: count, dtype: int64

Amount sample :
   Quantity  UnitPrice  Amount
0        12       0.00    0.00
1        12       0.00    0.00
2        12       0.00    0.00
3        48       0.00    0.00
4        24       0.00    0.00


In [ ]:
# Vérifier proportion Quantity == 0
n_q0 = (df['Quantity'] == 0).sum()
print(f"Lignes avec Quantity == 0 : {n_q0} / {len(df)} ({n_q0/len(df):.2%})")

# Décision : supprimer ces lignes (recommandé sauf si besoin particulier)
df = df[df['Quantity'] != 0].copy()
print("Shape après suppression Quantities==0 :", df.shape)


Lignes avec Quantity == 0 : 0 / 1067371 (0.00%)
Shape après suppression Quantities==0 : (1067371, 12)


In [ ]:
# Déduplication sur colonnes logiques
dedup_cols = [c for c in ['InvoiceNo','StockCode','Quantity','UnitPrice','InvoiceDate','CustomerID'] if c in df.columns]
before = len(df)
df = df.drop_duplicates(subset=dedup_cols)
after = len(df)
print(f"Déduplication : {before-after} lignes supprimées")


Déduplication : 35959 lignes supprimées


In [ ]:
# Flags outliers (99e percentile)
qty_q99 = df['Quantity'].abs().quantile(0.99)
price_q99 = df['UnitPrice'].abs().quantile(0.99)
df['is_qty_outlier'] = df['Quantity'].abs() > qty_q99
df['is_price_outlier'] = df['UnitPrice'].abs() > price_q99
df['is_outlier'] = df[['is_qty_outlier','is_price_outlier']].any(axis=1)

print("Outliers summary :")
print("Qty threshold (99%) :", qty_q99)
print("Price threshold (99%) :", price_q99)
print(df['is_outlier'].value_counts())
# montre quelques outliers
df.loc[df['is_outlier']].head(10)


Outliers summary :
Qty threshold (99%) : 120.0
Price threshold (99%) : 0.0
is_outlier
False    1021832
True        9580
Name: count, dtype: int64


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,__source_sheet,UnitPrice,is_cancel,Amount,is_qty_outlier,is_price_outlier,is_outlier
217,489460,84598,BOYS ALPHABET IRON ON PATCHES,576,2009-12-01 10:46:00,0.21,16167.00,United Kingdom,Year 2009-2010,0.00,False,0.00,True,False,True
223,489460,84568,GIRLS ALPHABET IRON ON PATCHES,288,2009-12-01 10:46:00,0.21,16167.00,United Kingdom,Year 2009-2010,0.00,False,0.00,True,False,True
282,489465,84879,ASSORTED COLOUR BIRD ORNAMENT,160,2009-12-01 10:52:00,1.45,13767.00,United Kingdom,Year 2009-2010,0.00,False,0.00,True,False,True
283,489463,71477,short,-240,2009-12-01 10:52:00,0.00,NaN,United Kingdom,Year 2009-2010,0.00,False,-0.00,True,False,True
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.00,NaN,United Kingdom,Year 2009-2010,0.00,False,-0.00,True,False,True
575,489523,84879,ASSORTED COLOUR BIRD ORNAMENT,800,2009-12-01 11:46:00,1.45,12931.00,United Kingdom,Year 2009-2010,0.00,False,0.00,True,False,True
934,489539,22086,PAPER CHAIN KIT 50'S CHRISTMAS,160,2009-12-01 12:18:00,2.55,15061.00,United Kingdom,Year 2009-2010,0.00,False,0.00,True,False,True
935,489539,22083,PAPER CHAIN KIT RETRO SPOT,160,2009-12-01 12:18:00,2.55,15061.00,United Kingdom,Year 2009-2010,0.00,False,0.00,True,False,True
937,489539,85040B,SET/4 BLUE FLOWER CANDLES IN BOWL,192,2009-12-01 12:18:00,1.45,15061.00,United Kingdom,Year 2009-2010,0.00,False,0.00,True,False,True
943,489539,21700,BIG DOUGHNUT FRIDGE MAGNETS,144,2009-12-01 12:18:00,0.85,15061.00,United Kingdom,Year 2009-2010,0.00,False,0.00,True,False,True


In [ ]:
if 'Country' in df.columns:
    df['Country'] = df['Country'].astype('string').str.strip().str.title()
    print("Top 10 pays après harmonisation :")
    print(df['Country'].value_counts().head(10))
else:
    print("Pas de colonne Country trouvée")


Top 10 pays après harmonisation :
Country
United Kingdom    946754
Eire               17656
Germany            17329
France             14018
Netherlands         5136
Spain               3749
Switzerland         3174
Belgium             3107
Portugal            2527
Australia           1890
Name: count, dtype: Int64


In [ ]:
# Version incluant retours
transactions_incl = df.copy()

# Version excluant retours
transactions_excl = df[~df['is_cancel']].copy()

# Split par présence CustomerID
if 'CustomerID' in transactions_excl.columns:
    transactions_customers = transactions_excl[transactions_excl['CustomerID'].notnull()].copy()
    transactions_no_customer = transactions_excl[transactions_excl['CustomerID'].isnull()].copy()
else:
    transactions_customers = transactions_excl.copy()
    transactions_no_customer = pd.DataFrame(columns=transactions_excl.columns)

print("Taille (incl returns) :", transactions_incl.shape)
print("Taille (excl returns) :", transactions_excl.shape)
print("Taille (customers only) :", transactions_customers.shape)
print("Taille (no customer) :", transactions_no_customer.shape)


Taille (incl returns) : (1031412, 15)
Taille (excl returns) : (1031412, 15)
Taille (customers only) : (1031412, 15)
Taille (no customer) : (0, 15)


In [ ]:
# Construire RFM pour customers
if transactions_customers.shape[0] == 0:
    print("Aucun transaction client trouvé : impossible de construire RFM")
else:
   snapshot = transactions_customers['InvoiceDate'].max() + pd.Timedelta(days=1)

cust_rfm = transactions_customers.groupby('Customer ID').agg(
    recency_days=('InvoiceDate', lambda x: int((snapshot - x.max()).days)),
    frequency=('Invoice', lambda x: x.nunique()),
    monetary=('Amount', 'sum'),
    first_order=('InvoiceDate', 'min'),
    last_order=('InvoiceDate', 'max'),
).reset_index()





In [ ]:
# Diagnostics rapides
print("Nb clients :", cust_rfm.shape[0])
print("Recency distribution (days) :")
print(cust_rfm['recency_days'].describe())
print("\nFrequency :")
print(cust_rfm['frequency'].describe())
print("\nMonetary :")
print(cust_rfm['monetary'].describe())

# Exemple : créer scores R/F/M en quintiles
cust_rfm['r_score'] = pd.qcut(cust_rfm['recency_days'].rank(method='first', ascending=False), 5, labels=False) + 1
cust_rfm['f_score'] = pd.qcut(cust_rfm['frequency'].rank(method='first', ascending=True), 5, labels=False) + 1
cust_rfm['m_score'] = pd.qcut(cust_rfm['monetary'].rank(method='first', ascending=True), 5, labels=False) + 1
cust_rfm['RFM'] = cust_rfm['r_score'].astype(str) + cust_rfm['f_score'].astype(str) + cust_rfm['m_score'].astype(str)

cust_rfm[['Customer ID','recency_days','frequency','monetary','RFM']].head()



Nb clients : 5936
Recency distribution (days) :
count   5936.00
mean     202.42
std      211.42
min        1.00
25%       25.00
50%       96.00
75%      381.00
max      739.00
Name: recency_days, dtype: float64

Frequency :
count   5936.00
mean       7.55
std       15.96
min        1.00
25%        2.00
50%        4.00
75%        8.00
max      508.00
Name: frequency, dtype: float64

Monetary :
count   5936.00
mean       0.00
std        0.00
min        0.00
25%        0.00
50%        0.00
75%        0.00
max        0.00
Name: monetary, dtype: float64


,Customer ID,recency_days,frequency,monetary,RFM
0,12346.00,326,17,0.00,251
1,12347.00,2,8,0.00,541
2,12348.00,75,5,0.00,331
3,12349.00,19,5,0.00,431
4,12350.00,310,1,0.00,211


In [ ]:
# Sauvegardes : Parquet + Excel + QC report


def save_df(df_obj, name):
    path_parquet = OUT_DIR / f"{name}.parquet"
    path_excel = OUT_DIR / f"{name}.xlsx"

    # Parquet
    df_obj.to_parquet(path_parquet, index=False)

    # Excel (propre, sans warnings zipfile)
    with pd.ExcelWriter(path_excel, engine="openpyxl") as writer:
        df_obj.to_excel(writer, index=False)

    print(f"Saved {name}: parquet -> {path_parquet}, excel -> {path_excel}")



for df in [transactions_incl, transactions_excl, transactions_customers]:
    df["Invoice"] = df["Invoice"].astype("string")

# Save transactions (incl + excl) and customers
save_df(transactions_incl, "transactions_including_returns")
save_df(transactions_excl, "transactions_excluding_returns")
save_df(transactions_customers, "transactions_customers")
save_df(cust_rfm, "customers_rfm")

# QC report
qc = {
    'initial_profile': profile0,
    'after_clean_rows': int(len(df)),
    'transactions_including_returns_rows': int(len(transactions_incl)),
    'transactions_excluding_returns_rows': int(len(transactions_excl)),
    'transactions_customers_rows': int(len(transactions_customers)),
    'n_customers': int(cust_rfm.shape[0]),
    'qty_outliers_count': int(df['is_qty_outlier'].sum()),
    'price_outliers_count': int(df['is_price_outlier'].sum())
}

qc_path = OUT_DIR / "qc_report.json"
with open(qc_path, 'w', encoding='utf-8') as f:
    json.dump(qc, f, indent=2, default=str)
print("QC report saved to", qc_path)

for col in transactions_excl.columns:
    print(col, transactions_excl[col].dtype, transactions_excl[col].apply(type).unique())



Saved transactions_including_returns: parquet -> clean_data\transactions_including_returns.parquet, excel -> clean_data\transactions_including_returns.xlsx
Saved transactions_excluding_returns: parquet -> clean_data\transactions_excluding_returns.parquet, excel -> clean_data\transactions_excluding_returns.xlsx
Saved transactions_customers: parquet -> clean_data\transactions_customers.parquet, excel -> clean_data\transactions_customers.xlsx
Saved customers_rfm: parquet -> clean_data\customers_rfm.parquet, excel -> clean_data\customers_rfm.xlsx
QC report saved to clean_data\qc_report.json
Invoice string [<class 'str'>]
StockCode string [<class 'str'>]
Description string [<class 'str'> <class 'pandas._libs.missing.NAType'>]
Quantity int64 [<class 'int'>]
InvoiceDate datetime64[ns] [<class 'pandas._libs.tslibs.timestamps.Timestamp'>]
Price float64 [<class 'float'>]
Customer ID float64 [<class 'float'>]
Country string [<class 'str'>]
__source_sheet object [<class 'str'>]
UnitPrice float64 [

# Interprétations & prochaines étapes

### Observations potentielles
- Taux de CustomerID manquants : (voir qc_report.json) — si élevé, réfléchir à l'impact sur RFM/CLV.
- Retours : nous avons produit une version excluant retours pour analyses CRM (recommandé pour RFM) mais gardé la version incluant retours pour transparency.
- Outliers : identifiés, non supprimés. À analyser article par article (StockCode).

### Recommandations pour l'app Streamlit
- Charger `transactions_excluding_returns.parquet` pour les dashboards RFM/cohortes de base.
- Offrir un commutateur `include_returns` si besoin d'étudier l'impact des retours.
- Mettre en place filtre pays et période glissante.


In [ ]:
import json

qc = {
    "nb_lignes": len(transactions_customers),
    "nb_colonnes": len(transactions_customers.columns),
    "nb_clients_uniques": transactions_customers["Customer ID"].nunique(),
    "nb_factures_uniques": transactions_customers["Invoice"].nunique(),
    "periode_min": str(transactions_customers["InvoiceDate"].min()),
    "periode_max": str(transactions_customers["InvoiceDate"].max()),
    "nb_missing_customer_id": transactions_customers["Customer ID"].isna().sum(),
    "nb_outliers": transactions_customers["is_outlier"].sum() if "is_outlier" in transactions_customers.columns else "N/A",
    "nb_retours": transactions_customers["is_cancel"].sum() if "is_cancel" in transactions_customers.columns else "N/A",
}

print("Résumé QC :")
qc = {k: (int(v) if isinstance(v, (np.integer,)) else
          float(v) if isinstance(v, (np.floating,)) else
          bool(v) if isinstance(v, (np.bool_,)) else
          v.isoformat() if hasattr(v, "isoformat") else
          v)
      for k, v in qc.items()}

print(json.dumps(qc, indent=2))
print("\nTop 5 clients par monetary :")



Résumé QC :
{
  "nb_lignes": 1031412,
  "nb_colonnes": 15,
  "nb_clients_uniques": 5936,
  "nb_factures_uniques": 53589,
  "periode_min": "2009-12-01 07:45:00",
  "periode_max": "2011-12-09 12:50:00",
  "nb_missing_customer_id": 234016,
  "nb_outliers": 9580,
  "nb_retours": 0
}

Top 5 clients par monetary :
